In [3]:
# ✅ 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 필수 라이브러리 설치
!pip install --quiet gspread oauth2client pandas

# ✅ 3. 라이브러리 임포트
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ✅ 4. 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# ✅ 5. 스프레드시트 열기
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit"
spreadsheet = gc.open_by_url(spreadsheet_url)
worksheet = spreadsheet.worksheet("Sheet1")

# ✅ 6. 전체 시트 데이터를 DataFrame으로 가져오기 (헤더 + 데이터 모두 포함)
all_data = worksheet.get_all_values()
max_cols = max(len(row) for row in all_data)
normalized_data = [row + [""] * (max_cols - len(row)) for row in all_data]
df = pd.DataFrame(normalized_data[1:], columns=normalized_data[0])  # 헤더: 0번째 행

# ✅ 7. 장소명 기준 주소/위도/경도/좌표 매핑
mapping = {
    "제주공항": ("제주시 공항로 2", 33.5056, 126.4929),
    "제주고기국수모던동네함봉본점": ("제주시 1100로 3029", 33.4806, 126.5191),
    "화흑돼지마을": ("제주시 조천읍 남조로 2455", 33.4654, 126.6244),
    "함덕해수욕장": ("제주시 조천읍 함덕리 1008", 33.5411, 126.6722),
    "우리두리우럭정식": ("제주시 구좌읍 김녕리 2761", 33.5602, 126.7671),
    "해비치 호텔": ("서귀포시 표선면 민속해안로 537", 33.3251, 126.8377),
    "한라산아래채정원": ("서귀포시 표선면 번영로 4714", 33.3333, 126.8322),
    "비자림": ("제주시 구좌읍 비자숲길 55", 33.5146, 126.7982),
    "어영이해녀": ("서귀포시 성산읍 일주동로 4150", 33.4683, 126.9256),
    "제주흑돼지명소 표선점": ("서귀포시 표선면 표선리 460-5", 33.3273, 126.8370),
    "휴애리자연생활공원": ("서귀포시 남원읍 신례동로 256", 33.2966, 126.6286),
    "공천포식당": ("서귀포시 남원읍 공천포로 89", 33.2631, 126.6644),
    "신화월드": ("서귀포시 안덕면 신화역사로304번길 38", 33.3033, 126.2887),
    "서녘돈까스": ("서귀포시 대정읍 영어도시로 74", 33.2204, 126.2563),
    "진미명가": ("서귀포시 대정읍 하모상가로 40", 33.2181, 126.2546),
    "도치돌아파카목장": ("서귀포시 표선면 가시리 산 22-4", 33.3385, 126.8082),
    "해적게": ("제주시 애월읍 애월해안로 256", 33.4593, 126.3112),
    "우영우": ("제주시 애월읍 고내로 10", 33.4698, 126.3201)
}

# ✅ 8. 매핑 함수 정의 및 적용
def get_location_info(place):
    if place in mapping:
        addr, lat, lng = mapping[place]
        return pd.Series([addr, lat, lng, f"{lat}, {lng}"])
    else:
        return pd.Series(["", "", "", ""])

df[["주소", "위도", "경도", "좌표"]] = df["장소"].apply(get_location_info)

# ✅ 9. 스프레드시트 업데이트
worksheet.update([df.columns.tolist()] + df.values.tolist())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


{'spreadsheetId': '1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs',
 'updatedRange': 'Sheet1!A1:M33',
 'updatedRows': 33,
 'updatedColumns': 13,
 'updatedCells': 429}

In [4]:
# ✅ 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 필수 라이브러리 설치
!pip install --quiet gspread oauth2client pandas

# ✅ 3. 라이브러리 임포트
import pandas as pd
import gspread
import numpy as np
from oauth2client.service_account import ServiceAccountCredentials

# ✅ 4. Google 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# ✅ 5. 스프레드시트 열기
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit"
spreadsheet = gc.open_by_url(spreadsheet_url)
worksheet = spreadsheet.worksheet("Sheet1")

# ✅ 6. 전체 데이터 로드
all_data = worksheet.get_all_values()
max_cols = max(len(row) for row in all_data)
normalized_data = [row + [""] * (max_cols - len(row)) for row in all_data]
df = pd.DataFrame(normalized_data[1:], columns=normalized_data[0])  # 첫 번째 행은 헤더

# ✅ 7. 장소 기준 주소/위도/경도 매핑
mapping = {
    "제주공항": ("제주시 공항로 2", 33.5056, 126.4929),
    "제주고기국수모던동네함봉본점": ("제주시 1100로 3029", 33.4806, 126.5191),
    "화흑돼지마을": ("제주시 조천읍 남조로 2455", 33.4654, 126.6244),
    "함덕해수욕장": ("제주시 조천읍 함덕리 1008", 33.5411, 126.6722),
    "우리두리우럭정식": ("제주시 구좌읍 김녕리 2761", 33.5602, 126.7671),
    "해비치 호텔": ("서귀포시 표선면 민속해안로 537", 33.3251, 126.8377),
    "한라산아래채정원": ("서귀포시 표선면 번영로 4714", 33.3333, 126.8322),
    "비자림": ("제주시 구좌읍 비자숲길 55", 33.5146, 126.7982),
    "어영이해녀": ("서귀포시 성산읍 일주동로 4150", 33.4683, 126.9256),
    "제주흑돼지명소 표선점": ("서귀포시 표선면 표선리 460-5", 33.3273, 126.8370),
    "휴애리자연생활공원": ("서귀포시 남원읍 신례동로 256", 33.2966, 126.6286),
    "공천포식당": ("서귀포시 남원읍 공천포로 89", 33.2631, 126.6644),
    "신화월드": ("서귀포시 안덕면 신화역사로304번길 38", 33.3033, 126.2887),
    "서녘돈까스": ("서귀포시 대정읍 영어도시로 74", 33.2204, 126.2563),
    "진미명가": ("서귀포시 대정읍 하모상가로 40", 33.2181, 126.2546),
    "도치돌아파카목장": ("서귀포시 표선면 가시리 산 22-4", 33.3385, 126.8082)
}

# ✅ 8. 매핑 적용 (주소, 위도, 경도, 좌표)
def get_location_info(place):
    if place in mapping:
        addr, lat, lng = mapping[place]
        return pd.Series([addr, lat, lng, f"{lat}, {lng}"])
    else:
        return pd.Series(["", "", "", ""])

df[["주소", "위도", "경도", "좌표"]] = df["장소"].apply(get_location_info)

# ✅ 9. 이동시간 및 거리 샘플 추가
df["예상시간(분)"] = np.random.randint(10, 50, size=len(df))
df["이동거리(km)"] = np.round(np.random.uniform(5, 40, size=len(df)), 1)

# ✅ 10. 구글 지도 링크 생성
df["지도링크"] = df["좌표"].apply(lambda x: f"https://www.google.com/maps/search/?api=1&query={x}")

# ✅ 11. 최종 결과 스프레드시트 업데이트
worksheet.update([df.columns.tolist()] + df.values.tolist())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


{'spreadsheetId': '1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs',
 'updatedRange': 'Sheet1!A1:P33',
 'updatedRows': 33,
 'updatedColumns': 16,
 'updatedCells': 528}

In [5]:
# ✅ 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 라이브러리 설치
!pip install --quiet gspread oauth2client pandas

# ✅ 3. 라이브러리 임포트
import pandas as pd
import gspread
import requests
import time
from oauth2client.service_account import ServiceAccountCredentials

# ✅ 4. 구글 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# ✅ 5. 시트 연결 및 데이터프레임 복원
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit"
spreadsheet = gc.open_by_url(spreadsheet_url)
worksheet = spreadsheet.worksheet("Sheet1")

all_data = worksheet.get_all_values()
max_cols = max(len(row) for row in all_data)
normalized_data = [row + [""] * (max_cols - len(row)) for row in all_data]
df = pd.DataFrame(normalized_data[1:], columns=normalized_data[0])

# ✅ 6. 카카오 API로 좌표 변환 함수 정의
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"

def get_coords_from_kakao(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        documents = response.json().get("documents")
        if documents:
            first_match = documents[0]
            lat = first_match["y"]
            lng = first_match["x"]
            return lat, lng
    return "", ""

# ✅ 7. 주소가 누락된 행에 대해 자동 좌표 보완
for idx, row in df.iterrows():
    if not row["주소"]:
        query = row["장소"]
        lat, lng = get_coords_from_kakao(query, KAKAO_API_KEY)
        if lat and lng:
            df.at[idx, "주소"] = query
            df.at[idx, "위도"] = lat
            df.at[idx, "경도"] = lng
            df.at[idx, "좌표"] = f"{lat}, {lng}"
        time.sleep(0.3)  # 과도한 호출 방지

# ✅ 8. 지도 링크 자동 생성
df["지도링크"] = df["좌표"].apply(lambda x: f"https://www.google.com/maps/search/?api=1&query={x}" if x else "")

# ✅ 9. 결과 반영
worksheet.update([df.columns.tolist()] + df.values.tolist())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


{'spreadsheetId': '1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs',
 'updatedRange': 'Sheet1!A1:P33',
 'updatedRows': 33,
 'updatedColumns': 16,
 'updatedCells': 528}

In [6]:
# prompt: from google.colab import drive
# drive.mount('/content/drive')
# !pip install --quiet gspread oauth2client pandas
# import pandas as pd
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials
# # 인증
# scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
# credentials = ServiceAccountCredentials.from_json_keyfile_name(
#     "/content/drive/MyDrive/Key/credentials.json", scope)
# gc = gspread.authorize(credentials)
# # 시트 열기
# spreadsheet_url = "https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit"
# spreadsheet = gc.open_by_url(spreadsheet_url)
# worksheet = spreadsheet.worksheet("Sheet1")
# # 데이터프레임으로 변환
# all_data = worksheet.get_all_values()
# max_cols = max(len(row) for row in all_data)
# normalized_data = [row + [""] * (max_cols - len(row)) for row in all_data]
# df = pd.DataFrame(normalized_data[1:], columns=normalized_data[0])
# # 주소가 누락된 장소 확인
# missing_places = df[df["주소"] == ""]["장소"].unique().tolist()
# print("📌 주소 누락 장소 목록:")
# print(missing_places)

from google.colab import drive
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
import time

# Mount Google Drive
drive.mount('/content/drive')

# Install necessary libraries
!pip install --quiet gspread oauth2client pandas

# Authenticate with Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# Open the spreadsheet and worksheet
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit"
spreadsheet = gc.open_by_url(spreadsheet_url)
worksheet = spreadsheet.worksheet("Sheet1")

# Load data into a pandas DataFrame
all_data = worksheet.get_all_values()
max_cols = max(len(row) for row in all_data)
normalized_data = [row + [""] * (max_cols - len(row)) for row in all_data]
df = pd.DataFrame(normalized_data[1:], columns=normalized_data[0])

# Kakao API Key (replace with your actual key)
KAKAO_API_KEY = "YOUR_KAKAO_API_KEY"  #  Replace with your actual API key

def get_coords_from_kakao(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        result = response.json()
        if 'documents' in result and result['documents']:
            return result['documents'][0]['y'], result['documents'][0]['x']
    return None, None

# Update missing addresses using Kakao API
for index, row in df.iterrows():
    if not row["주소"]:
        place = row['장소']
        latitude, longitude = get_coords_from_kakao(place, KAKAO_API_KEY)

        if latitude and longitude:
            df.loc[index, "주소"] = place # Update 주소 with the place name if coordinates are found.
            df.loc[index, "위도"] = latitude
            df.loc[index, "경도"] = longitude
            df.loc[index, "좌표"] = f"{latitude}, {longitude}"
        else:
            print(f"Coordinates not found for '{place}'") # Print message if not found.

        time.sleep(0.3)  # Rate limiting to prevent exceeding API limits

# Update the Google Sheet with the new data
worksheet.update([df.columns.tolist()] + df.values.tolist())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Coordinates not found for '제주고기국수모던돔베공항본점'
Coordinates not found for '와흘메밀마을'
Coordinates not found for '함덕해수욕장 '
Coordinates not found for '해비치 호텔 조식'
Coordinates not found for '민속촌 및 해비치호텔둘레길,키드존'
Coordinates not found for '한라산아래첫마을 '
Coordinates not found for '어멍이해녀'
Coordinates not found for '제주할매밥상 표선점'
Coordinates not found for '키즈존 및 체크 아웃'
Coordinates not found for '큰엉해안경승지(25분)'
Coordinates not found for '탐나는 식당 (35분)'
Coordinates not found for '휴애리자연생활공원(9분)'
Coordinates not found for '야크마을'
Coordinates not found for '큰돈가'
Coordinates not found for '조식 (3인추가 가격내야함)'
Coordinates not found for '산양큰엉곶'
Coordinates not found for '춘미향'
Coordinates not found for '도치돌알파카 목장 (25분)'
Coordinates not found for '조식 (3인추가 가격내야함)'
Coordinates not found for '체크아웃'
Coordinates not found for '제주돔베막국수'
Coordinates not found for '함덕해수욕장 '
Coordinates not found for '해지

{'spreadsheetId': '1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs',
 'updatedRange': 'Sheet1!A1:P33',
 'updatedRows': 33,
 'updatedColumns': 16,
 'updatedCells': 528}

In [14]:
# ✅ 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 필요한 패키지 설치
!pip install --quiet gspread oauth2client pandas

# ✅ 3. 필수 라이브러리 임포트
import pandas as pd
import gspread
import requests
import time
from oauth2client.service_account import ServiceAccountCredentials

# ✅ 4. 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# ✅ 5. 카카오 API 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"

def get_coords_from_kakao(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        documents = response.json().get("documents")
        if documents:
            first_match = documents[0]
            lat = first_match["y"]
            lng = first_match["x"]
            return lat, lng
    return "", ""

# ✅ 6. 스프레드시트 열기
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit"
spreadsheet = gc.open_by_url(spreadsheet_url)
worksheet = spreadsheet.worksheet("Sheet1")

# ✅ 7. 데이터프레임 로딩
all_data = worksheet.get_all_values()
max_cols = max(len(row) for row in all_data)
normalized_data = [row + [""] * (max_cols - len(row)) for row in all_data]
df = pd.DataFrame(normalized_data[1:], columns=normalized_data[0])

# ✅ 8. 장소명 정제 매핑
manual_place_corrections = {
    "제주고기국수모던돔베공항본점": "제주고기국수 공항점",
    "와흘메밀마을": "와흘메밀마을",
    "함덕해수욕장 ": "함덕해수욕장",
    "해비치 호텔 조식": "해비치 호텔",
    "민속촌 및 해비치호텔둘레길,키드존": "해비치호텔",
    "한라산아래첫마을 ": "한라산아래첫마을",
    "어멍이해녀": "성산 어멍이해녀",
    "제주할매밥상 표선점": "제주할매밥상 표선점",
    "키즈존 및 체크 아웃": "해비치 호텔",
    "큰엉해안경승지(25분)": "큰엉해안경승지",
    "탐나는 식당 (35분)": "탐나는식당 서귀포",
    "휴애리자연생활공원(9분)": "휴애리자연생활공원",
    "야크마을": "야크마을 카페",
    "큰돈가": "큰돈가 흑돼지",
    "조식 (3인추가 가격내야함)": "해비치 호텔 조식당",
    "산양큰엉곶": "산양큰엉곶 전망대",
    "춘미향": "춘미향 음식점",
    "도치돌알파카 목장 (25분)": "도치돌 알파카 목장",
    "체크아웃": "해비치 호텔",
    "제주돔베막국수": "제주 돔베 막국수",
    "해지개": "애월 해지개 카페",
    "우영담": "우영담 카페",
    "렌트카 반납": "제주 렌트카하우스",
    "공항": "제주공항"
}

# ✅ 9. 정제된 장소 기준으로 좌표 보완
df["장소_보정"] = df["장소"].apply(lambda x: manual_place_corrections.get(x.strip(), x.strip()))
df["주소"] = df["주소"].fillna("")

for idx, row in df.iterrows():
    if not row["주소"]:
        query = row["장소_보정"]
        lat, lng = get_coords_from_kakao(query, KAKAO_API_KEY)
        if lat and lng:
            df.at[idx, "주소"] = query
            df.at[idx, "위도"] = lat
            df.at[idx, "경도"] = lng
            df.at[idx, "좌표"] = f"{lat}, {lng}"
        time.sleep(0.3)

df["지도링크"] = df["좌표"].apply(lambda x: f"https://www.google.com/maps/search/?api=1&query={x}" if x else "")
worksheet.update([df.columns.tolist()] + df.values.tolist())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


{'spreadsheetId': '1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs',
 'updatedRange': 'Sheet1!A1:O32',
 'updatedRows': 32,
 'updatedColumns': 15,
 'updatedCells': 480}

In [15]:
# ✅ 1. 드라이브 마운트 및 라이브러리 설치
from google.colab import drive
drive.mount('/content/drive')

!pip install --quiet gspread oauth2client pandas

import pandas as pd
import gspread
import requests
import time
from oauth2client.service_account import ServiceAccountCredentials

# ✅ 2. 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# ✅ 3. 카카오 API 함수
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"
def get_coords_from_kakao(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        documents = response.json().get("documents")
        if documents:
            first_match = documents[0]
            lat = first_match["y"]
            lng = first_match["x"]
            return lat, lng
    return "", ""

# ✅ 4. 스프레드시트 및 시트 불러오기
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit"
spreadsheet = gc.open_by_url(spreadsheet_url)
worksheet = spreadsheet.worksheet("Sheet1")

# ✅ 5. 시트 데이터프레임으로 로딩
all_data = worksheet.get_all_values()
max_cols = max(len(row) for row in all_data)
normalized_data = [row + [""] * (max_cols - len(row)) for row in all_data]
df = pd.DataFrame(normalized_data[1:], columns=normalized_data[0])

# ✅ 6. 장소 보정 맵 적용
manual_place_corrections = {
    "제주고기국수모던돔베공항본점": "제주고기국수 공항점",
    "와흘메밀마을": "와흘메밀마을",
    "함덕해수욕장 ": "함덕해수욕장",
    "해비치 호텔 조식": "해비치 호텔",
    "민속촌 및 해비치호텔둘레길,키드존": "해비치호텔",
    "한라산아래첫마을 ": "한라산아래첫마을",
    "어멍이해녀": "성산 어멍이해녀",
    "제주할매밥상 표선점": "제주할매밥상 표선점",
    "키즈존 및 체크 아웃": "해비치 호텔",
    "큰엉해안경승지(25분)": "큰엉해안경승지",
    "탐나는 식당 (35분)": "탐나는식당 서귀포",
    "휴애리자연생활공원(9분)": "휴애리자연생활공원",
    "야크마을": "야크마을 카페",
    "큰돈가": "큰돈가 흑돼지",
    "조식 (3인추가 가격내야함)": "해비치 호텔 조식당",
    "산양큰엉곶": "산양큰엉곶 전망대",
    "춘미향": "춘미향 음식점",
    "도치돌알파카 목장 (25분)": "도치돌 알파카 목장",
    "체크아웃": "해비치 호텔",
    "제주돔베막국수": "제주 돔베 막국수",
    "해지개": "애월 해지개 카페",
    "우영담": "우영담 카페",
    "렌트카 반납": "제주 렌트카하우스",
    "공항": "제주공항"
}

df["장소_보정"] = df["장소"].apply(lambda x: manual_place_corrections.get(x.strip(), x.strip()))
df["주소"] = df["주소"].fillna("")

# ✅ 7. 좌표, 지도링크 보완
for idx, row in df.iterrows():
    if not row["주소"]:
        query = row["장소_보정"]
        lat, lng = get_coords_from_kakao(query, KAKAO_API_KEY)
        if lat and lng:
            df.at[idx, "주소"] = query
            df.at[idx, "위도"] = lat
            df.at[idx, "경도"] = lng
            df.at[idx, "좌표"] = f"{lat}, {lng}"
        time.sleep(0.3)

df["지도링크"] = df["좌표"].apply(lambda x: f"https://www.google.com/maps/search/?api=1&query={x}" if x else "")

# ✅ 8. 결과 반영
worksheet.update([df.columns.tolist()] + df.values.tolist())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


{'spreadsheetId': '1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs',
 'updatedRange': 'Sheet1!A1:O32',
 'updatedRows': 32,
 'updatedColumns': 15,
 'updatedCells': 480}

In [9]:
# ✅ 데이터프레임 다시 복원
all_data = worksheet.get_all_values()
max_cols = max(len(row) for row in all_data)
normalized_data = [row + [""] * (max_cols - len(row)) for row in all_data]
df = pd.DataFrame(normalized_data[1:], columns=normalized_data[0])

# ✅ 누락된 주소 행 확인
incomplete_df = df[df["주소"].isnull() | (df["주소"] == "")]
incomplete_places = incomplete_df["장소_보정"].unique().tolist()
incomplete_places


['제주고기국수 공항점',
 '와흘메밀마을',
 '함덕해수욕장',
 '해비치 호텔',
 '해비치호텔',
 '한라산아래첫마을',
 '성산 어멍이해녀',
 '제주할매밥상 표선점',
 '큰엉해안경승지',
 '탐나는식당 서귀포',
 '휴애리자연생활공원',
 '야크마을 카페',
 '큰돈가 흑돼지',
 '해비치 호텔 조식당',
 '산양큰엉곶 전망대',
 '춘미향 음식점',
 '도치돌 알파카 목장',
 '제주 돔베 막국수',
 '애월 해지개 카페',
 '우영담 카페',
 '제주 렌트카하우스',
 '제주공항',
 '']

In [12]:
# ✅ 필요한 라이브러리 다시 임포트
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ✅ 인증 재설정
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# ✅ 스프레드시트 열기
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit"
spreadsheet = gc.open_by_url(spreadsheet_url)
worksheet = spreadsheet.worksheet("Sheet1")


In [13]:
# ✅ 백업 시트가 있을 경우 복원
backup_worksheet = spreadsheet.worksheet("백업_자동생성")
backup_data = backup_worksheet.get_all_values()
worksheet.update(backup_data)


{'spreadsheetId': '1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs',
 'updatedRange': 'Sheet1!A1:Q33',
 'updatedRows': 33,
 'updatedColumns': 17,
 'updatedCells': 561}

In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# ✅ 주소(G열 기준)로 위도, 경도, 좌표, 지도링크 적용하는 전체 코드 (런타임 재시작 고려)

import pandas as pd
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests

# ✅ Kakao API 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"

# ✅ 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# ✅ 스프레드시트 열기
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit"
spreadsheet = gc.open_by_url(spreadsheet_url)
worksheet = spreadsheet.worksheet("Sheet1")

# ✅ 데이터프레임으로 불러오기
all_data = worksheet.get_all_values()
max_cols = max(len(row) for row in all_data)
normalized_data = [row + [""] * (max_cols - len(row)) for row in all_data]
df = pd.DataFrame(normalized_data[1:], columns=normalized_data[0])

# ✅ Kakao API로 좌표 검색 함수
def get_coords_from_kakao(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        documents = response.json().get("documents")
        if documents:
            lat = documents[0]["y"]
            lng = documents[0]["x"]
            return lat, lng
    return "", ""

# ✅ 주소(G열) 기준으로 위도, 경도, 좌표, 지도링크 갱신
for idx, row in df.iterrows():
    address = row["주소"]
    if address and (not row.get("위도") or not row.get("경도")):
        lat, lng = get_coords_from_kakao(address, KAKAO_API_KEY)
        if lat and lng:
            df.at[idx, "위도"] = lat
            df.at[idx, "경도"] = lng
            df.at[idx, "좌표"] = f"{lat}, {lng}"
            df.at[idx, "지도링크"] = f"https://www.google.com/maps/search/?api=1&query={lat},{lng}"
        time.sleep(0.3)

# ✅ 반영
worksheet.update([df.columns.tolist()] + df.values.tolist())


{'spreadsheetId': '1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs',
 'updatedRange': 'Sheet1!A1:N33',
 'updatedRows': 33,
 'updatedColumns': 14,
 'updatedCells': 462}

In [20]:
import pandas as pd
import time
import requests

KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"

# 좌표 기반 거리/시간 함수
def get_travel_info(from_coords, to_coords, api_key):
    url = "https://apis-navi.kakaomobility.com/v1/directions"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {
        "origin": f"{from_coords[1]},{from_coords[0]}",
        "destination": f"{to_coords[1]},{to_coords[0]}",
        "priority": "RECOMMEND"
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        try:
            route = response.json()["routes"][0]["summary"]
            return int(route["duration"] / 60), round(route["distance"] / 1000, 1)
        except:
            return "", ""
    return "", ""

# 데이터프레임 로딩
all_data = worksheet.get_all_values()
max_cols = max(len(row) for row in all_data)
normalized_data = [row + [""] * (max_cols - len(row)) for row in all_data]
df = pd.DataFrame(normalized_data[1:], columns=normalized_data[0])

# 정렬 (날짜/시간은 그대로 유지)
df["__날짜"] = pd.to_datetime(df["날짜"], errors='coerce')
df["__시간"] = pd.to_datetime(df["시간"], format="%H:%M", errors='coerce').dt.time
df_sorted = df.sort_values(by=["__날짜", "__시간"]).reset_index(drop=True)

# 이동 계산
travel_times = [""]
travel_dists = [""]

for i in range(1, len(df_sorted)):
    prev_row = df_sorted.iloc[i - 1]
    curr_row = df_sorted.iloc[i]
    if prev_row.get("위도") and prev_row.get("경도") and curr_row.get("위도") and curr_row.get("경도"):
        from_coords = (prev_row["위도"], prev_row["경도"])
        to_coords = (curr_row["위도"], curr_row["경도"])
        duration, distance = get_travel_info(from_coords, to_coords, KAKAO_API_KEY)
        travel_times.append(duration)
        travel_dists.append(distance)
        time.sleep(0.3)
    else:
        travel_times.append("")
        travel_dists.append("")

# 값 반영
df_sorted["이동시간(분)"] = travel_times
df_sorted["거리(km)"] = travel_dists

# 정렬용 임시 컬럼 제거
df_sorted.drop(columns=["__날짜", "__시간"], inplace=True)

# 시트 반영
worksheet.update([df_sorted.columns.tolist()] + df_sorted.values.tolist())


<ipython-input-20-725ae4667fed>:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["__날짜"] = pd.to_datetime(df["날짜"], errors='coerce')


{'spreadsheetId': '1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs',
 'updatedRange': 'Sheet1!A1:P33',
 'updatedRows': 33,
 'updatedColumns': 16,
 'updatedCells': 528}

In [23]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# 스프레드시트 열기
spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit")
worksheet = spreadsheet.worksheet("Sheet1")


In [24]:
# ✅ Google Sheets → DataFrame 불러오기
all_data = worksheet.get_all_values()
max_cols = max(len(row) for row in all_data)
normalized_data = [row + [""] * (max_cols - len(row)) for row in all_data]
df = pd.DataFrame(normalized_data[1:], columns=normalized_data[0])


In [25]:
# ✅ 날짜/시간 기준 내림차순 정렬 후 거리/시간 계산
df["__날짜"] = pd.to_datetime(df["날짜"], errors='coerce')
df["__시간"] = pd.to_datetime(df["시간"], format="%H:%M", errors='coerce').dt.time
df_sorted = df.sort_values(by=["__날짜", "__시간"], ascending=[False, False]).reset_index(drop=True)

travel_times = [""]
travel_dists = [""]

for i in range(1, len(df_sorted)):
    prev_row = df_sorted.iloc[i - 1]
    curr_row = df_sorted.iloc[i]
    if prev_row.get("위도") and prev_row.get("경도") and curr_row.get("위도") and curr_row.get("경도"):
        from_coords = (prev_row["위도"], prev_row["경도"])
        to_coords = (curr_row["위도"], curr_row["경도"])
        duration, distance = get_travel_info(from_coords, to_coords, KAKAO_API_KEY)
        travel_times.append(duration)
        travel_dists.append(distance)
        time.sleep(0.3)
    else:
        travel_times.append("")
        travel_dists.append("")

df_sorted["이동시간(분)"] = travel_times
df_sorted["거리(km)"] = travel_dists
df_sorted.drop(columns=["__날짜", "__시간"], inplace=True)

# Google Sheet 반영
worksheet.update([df_sorted.columns.tolist()] + df_sorted.values.tolist())


<ipython-input-25-98150ce5ac89>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["__날짜"] = pd.to_datetime(df["날짜"], errors='coerce')


{'spreadsheetId': '1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs',
 'updatedRange': 'Sheet1!A1:N33',
 'updatedRows': 33,
 'updatedColumns': 14,
 'updatedCells': 462}

In [6]:
# ✅ Google Colab에서 실행되는 전체 통합 코드 (Mac북 사용자 기준)

# 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. 필수 라이브러리 불러오기
import pandas as pd
import time
import gspread
import requests
from oauth2client.service_account import ServiceAccountCredentials

# 3. Kakao API Key
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"

# 4. Google Sheets 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# 5. 스프레드시트 열기
spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit")
worksheet = spreadsheet.worksheet("Sheet1")

# 6. 데이터 로드
all_data = worksheet.get_all_values()
max_cols = max(len(row) for row in all_data)
normalized_data = [row + [""] * (max_cols - len(row)) for row in all_data]
df = pd.DataFrame(normalized_data[1:], columns=normalized_data[0])

# 7. Kakao 주소 → 좌표 변환 함수
def get_coords_from_kakao(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        documents = response.json().get("documents")
        if documents:
            lat = documents[0]["y"]
            lng = documents[0]["x"]
            return lat, lng
    return "", ""

# 8. Kakao 길찾기 → 거리 및 시간 계산 함수
def get_travel_info(from_coords, to_coords, api_key):
    url = "https://apis-navi.kakaomobility.com/v1/directions"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {
        "origin": f"{from_coords[1]},{from_coords[0]}",
        "destination": f"{to_coords[1]},{to_coords[0]}",
        "priority": "RECOMMEND"
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        try:
            route = response.json()["routes"][0]["summary"]
            return int(route["duration"] / 60), round(route["distance"] / 1000, 1)
        except:
            return "", ""
    return "", ""

# 9. 주소 기반 위도/경도/좌표/지도링크 적용
for idx, row in df.iterrows():
    if not row.get("위도") or not row.get("경도"):
        address = row.get("주소")
        if address:
            lat, lng = get_coords_from_kakao(address, KAKAO_API_KEY)
            if lat and lng:
                df.at[idx, "위도"] = lat
                df.at[idx, "경도"] = lng
                df.at[idx, "좌표"] = f"{lat}, {lng}"
                df.at[idx, "지도링크"] = f"https://www.google.com/maps/search/?api=1&query={lat},{lng}"
            time.sleep(0.3)

# 10. A열(날짜), B열(시간) 기준 오름차순 정렬
df["__날짜"] = pd.to_datetime(df[df.columns[0]], errors='coerce')
df["__시간"] = pd.to_datetime(df[df.columns[1]], format="%H:%M", errors='coerce').dt.time
df_sorted = df.sort_values(by=["__날짜", "__시간"], ascending=[True, True]).reset_index(drop=True)

# 11. 이전 행과의 거리/시간 계산
travel_times = [""]
travel_dists = [""]

for i in range(1, len(df_sorted)):
    prev = df_sorted.iloc[i - 1]
    curr = df_sorted.iloc[i]
    if prev.get("위도") and prev.get("경도") and curr.get("위도") and curr.get("경도"):
        from_coords = (prev["위도"], prev["경도"])
        to_coords = (curr["위도"], curr["경도"])
        duration, distance = get_travel_info(from_coords, to_coords, KAKAO_API_KEY)
        travel_times.append(duration)
        travel_dists.append(distance)
        time.sleep(0.3)
    else:
        travel_times.append("")
        travel_dists.append("")

# 12. 최종 결과 반영
df_sorted["이동시간(분)"] = travel_times
df_sorted["거리(km)"] = travel_dists
df_sorted.drop(columns=["__날짜", "__시간"], inplace=True)

# 13. 스프레드시트에 반영
worksheet.update([df_sorted.columns.tolist()] + df_sorted.values.tolist())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-6-4bffe2348654>:79: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["__날짜"] = pd.to_datetime(df[df.columns[0]], errors='coerce')


{'spreadsheetId': '1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs',
 'updatedRange': 'Sheet1!A1:N33',
 'updatedRows': 33,
 'updatedColumns': 14,
 'updatedCells': 462}

In [31]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit")
worksheet = spreadsheet.worksheet("Sheet1")


In [32]:
# ✅ Google Sheets 재연결
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)
worksheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit").worksheet("Sheet1")

# ✅ 시트 데이터 불러오기
data = worksheet.get_all_values()
columns = data[0]
df = pd.DataFrame(data[1:], columns=columns)

# ✅ 날짜 및 시간 파싱
df["__날짜"] = pd.to_datetime(df["날짜"].str.replace("월 ", "-").apply(lambda x: f"2025-{x}"), errors='coerce')
df["__시간"] = pd.to_datetime(df["시간"], format="%H:%M", errors='coerce')

# ✅ 날짜+시간 오름차순 정렬
df_sorted = df.sort_values(by=["__날짜", "__시간"], ascending=[True, True]).reset_index(drop=True)
df_sorted.drop(columns=["__날짜", "__시간"], inplace=True)

# ✅ 시트 업데이트
worksheet.update([df_sorted.columns.tolist()] + df_sorted.values.tolist())


<ipython-input-32-c800450f4069>:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["__날짜"] = pd.to_datetime(df["날짜"].str.replace("월 ", "-").apply(lambda x: f"2025-{x}"), errors='coerce')


{'spreadsheetId': '1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs',
 'updatedRange': 'Sheet1!A1:N33',
 'updatedRows': 33,
 'updatedColumns': 14,
 'updatedCells': 462}

In [37]:
# 🔄 Google Sheets 인증 및 연결
from oauth2client.service_account import ServiceAccountCredentials
import gspread

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit")
worksheet = spreadsheet.worksheet("Sheet1")


In [35]:
import pandas as pd

# 시트 데이터 불러오기
data = worksheet.get_all_values()
columns = data[0]
df = pd.DataFrame(data[1:], columns=columns)

# 날짜 및 시간 파싱
df["__날짜"] = pd.to_datetime(df["날짜"].str.replace("월 ", "-").apply(lambda x: f"2025-{x}"), errors='coerce')
df["__시간"] = pd.to_datetime(df["시간"], format="%H:%M", errors='coerce')

# 07:00 이후 필터링
df = df[df["__시간"].dt.hour >= 7]

# 날짜 + 시간 오름차순 정렬
df_sorted = df.sort_values(by=["__날짜", "__시간"], ascending=[True, True]).reset_index(drop=True)
df_sorted.drop(columns=["__날짜", "__시간"], inplace=True)

# 시트 업데이트
worksheet.update([df_sorted.columns.tolist()] + df_sorted.values.tolist())


<ipython-input-35-230054ff7d58>:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["__날짜"] = pd.to_datetime(df["날짜"].str.replace("월 ", "-").apply(lambda x: f"2025-{x}"), errors='coerce')


{'spreadsheetId': '1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs',
 'updatedRange': 'Sheet1!A1:N33',
 'updatedRows': 33,
 'updatedColumns': 14,
 'updatedCells': 462}

In [36]:
# 런타임 리셋됨: Google Drive 재마운트 및 인증 코드부터 다시 시작

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# 🔄 Google Sheets 인증 및 연결
from oauth2client.service_account import ServiceAccountCredentials
import gspread

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit")
worksheet = spreadsheet.worksheet("Sheet1")


In [40]:
!pip install gspread oauth2client --quiet


In [42]:
!pip install oauth2client gspread --quiet


In [4]:
# ✅ 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 필수 패키지 설치
!pip install gspread oauth2client --quiet

# ✅ 3. 라이브러리 불러오기
import pandas as pd
import gspread
import requests
import time
from oauth2client.service_account import ServiceAccountCredentials

# ✅ 4. Google Sheets 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# ✅ 5. 시트 연결
spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1m64XJInJT0lzdg-POeo8ZmX1bZpEBVuX5or3EtMlVBs/edit")
worksheet = spreadsheet.worksheet("Sheet1")

# ✅ 6. 데이터 불러오기
data = worksheet.get_all_values()
cols = data[0]
df = pd.DataFrame(data[1:], columns=cols)

# ✅ 7. 날짜 및 시간 정렬
df["__날짜"] = pd.to_datetime(df["날짜"].str.replace("월 ", "-").str.replace("일", ""), format="%m-%d", errors='coerce')
df["__날짜"] = df["__날짜"].apply(lambda d: d.replace(year=2025) if pd.notnull(d) else d)
df["__시간"] = pd.to_datetime(df["시간"], format="%H:%M", errors='coerce')
df = df.sort_values(by=["__날짜", "__시간"]).reset_index(drop=True)
df.drop(columns=["__날짜", "__시간"], inplace=True)

# ✅ 8. Kakao 길찾기 API 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"

def get_travel_info(from_coords, to_coords):
    url = "https://apis-navi.kakaomobility.com/v1/directions"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {
        "origin": f"{from_coords[1]},{from_coords[0]}",
        "destination": f"{to_coords[1]},{to_coords[0]}",
        "priority": "RECOMMEND"
    }
    try:
        res = requests.get(url, headers=headers, params=params)
        if res.status_code == 200:
            route = res.json()["routes"][0]["summary"]
            return int(route["duration"] / 60), round(route["distance"] / 1000, 1)
    except:
        return "", ""
    return "", ""

# ✅ 9. 거리/시간 계산
durations = [""]
distances = [""]

for i in range(1, len(df)):
    try:
        from_lat, from_lng = float(df.loc[i-1, "위도"]), float(df.loc[i-1, "경도"])
        to_lat, to_lng = float(df.loc[i, "위도"]), float(df.loc[i, "경도"])
        duration, distance = get_travel_info((from_lat, from_lng), (to_lat, to_lng))
        durations.append(duration)
        distances.append(distance)
        time.sleep(0.3)
    except:
        durations.append("")
        distances.append("")

df["이동시간(분)"] = durations
df["거리(km)"] = distances

# ✅ 10. 열 정리
order = ["날짜", "시간", "장소", "주소", "이동시간(분)", "거리(km)", "좌표"]
others = [c for c in df.columns if c not in order]
df = df[order + others]

# ✅ 11. 저장 및 출력
df.to_csv("/content/제주도_최종일정.csv", index=False)
print("✅ 최종 CSV 저장 완료: /content/제주도_최종일정.csv")
df.head(10)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 최종 CSV 저장 완료: /content/제주도_최종일정.csv


,날짜,시간,장소,주소,이동시간(분),거리(km),좌표,구분,장소_기존,기타,위도,경도,비용,지도링크
0,5월 16일,10:10,제주공항,제주특별자치도 제주시 공항로 2,,,"33.5056, 126.4929",제주도착,제주공항,,33.51,126.49,,https://www.google.com/maps/search/?api=1&quer...
1,5월 16일,12:00,로그인렌트카,제주 제주시 다호북길 111,11,6.1,"33.502688128534, 126.486531671174",차량렌트,로그인렌트카,렌트,33.50,126.49,"₩ 280,000",https://www.google.com/maps/search/?api=1&quer...
2,5월 16일,12:30,제주고기국수 공항점,제주특별자치도 제주시 다호북길 109 1층,,,"33.5028594289933, 126.486770648605",점심,제주고기국수모던돔베공항본점,고기국수,33.50,126.49,"₩ 100,000",https://www.google.com/maps/search/?api=1&quer...
3,5월 16일,14:00,와흘메밀마을,제주 제주시 조천읍 남조로 2455,34,20.5,"33.4827833506604, 126.650068827028",사진촬영,와흘메밀마을,꽃구경,33.48,126.65,,https://www.google.com/maps/search/?api=1&quer...
4,5월 16일,16:00,함덕해수욕장,제주특별자치도 제주시 조천읍 조함해안로 525,19,10.8,"33.5428267548889, 126.670510321052",사진촬영,함덕해수욕장,바다구경,33.54,126.67,,https://www.google.com/maps/search/?api=1&quer...
5,5월 16일,18:00,우리두리우럭정식,제주시 구좌읍 김녕리 2761,19,11.3,"33.5602, 126.7671",저녁,우리두리우럭정식,우럭정식,33.56,126.77,"₩ 100,000",https://www.google.com/maps/search/?api=1&quer...
6,5월 16일,19:00,해비치 호텔,서귀포시 표선면 민속해안로 537,56,37.4,"33.3251, 126.8377",숙소,해비치 호텔,,33.33,126.84,,https://www.google.com/maps/search/?api=1&quer...
7,5월 17일,7:00,해비치 호텔,제주특별자치도 서귀포시 표선면 민속해안로 537,8,3.3,"33.3225989591113, 126.843977586306",아침,해비치 호텔 조식,호텔조식,33.32,126.84,"₩ 260,000",https://www.google.com/maps/search/?api=1&quer...
8,5월 17일,10:00,해비치호텔,제주특별자치도 서귀포시 표선면 민속해안로 537,,,"33.3225989591113, 126.843977586306",카페,"민속촌 및 해비치호텔둘레길,키드존",호텔커피,33.32,126.84,"₩ 40,000",https://www.google.com/maps/search/?api=1&quer...
9,5월 17일,13:30,한라산아래첫마을,제주 서귀포시 표선면 민속해안로 631-46,71,54.2,"33.3280820649176, 126.37968156946",점심,한라산아래첫마을,메밀면 기타등,33.33,126.38,"₩ 70,000",https://www.google.com/maps/search/?api=1&quer...


In [3]:
# ✅ 12. Google Sheets에 거리/시간 반영
from gspread_dataframe import set_with_dataframe

# 기존 데이터프레임 df를 기준으로 업데이트
set_with_dataframe(worksheet, df)
print("✅ 스프레드시트에 거리 및 이동시간 반영 완료")

✅ 스프레드시트에 거리 및 이동시간 반영 완료
